In [1]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  
import configparser

config = configparser.ConfigParser()
config.read('XXX-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

rooms=config.get('Configuration', 'rooms')
rooms = [(item.strip()) for item in rooms.split(',')]

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))

user_color={}
for user in users:
    user_color[user]=config.get('Colors', user)
print(user_color)


{'f02e': 'blue', '867f': 'red', 'ed9c': 'green'}


In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

day=list(range(day_time(t0),day_time(tN)+1))[0]
day=19798
print(day)

1980-01-01 13:00:02 Init date is 315576002 day: 3652 1980-01-01 13:00:02
1980-01-01 14:51:20 End date is 315582680 day: 3652 1980-01-01 14:51:20
19798


In [3]:
i0=0
ts=list(range(t0,tN,1))
#print(i0,ts)

loc_sensor={}

def getLoc(room,s,oH,oW):
    print("S:",s)
    xy=config.get(room, s)
    xy = [(item.strip()) for item in xy.split(',')]
    xy = [list(map(int, item.split('x'))) for item in xy]
    return [xy[0][0]+oH,xy[0][1]+oW]

offW= {}
offH={}

for room in rooms:
    print(room)

    offW[room] = int(config.get(room, 'offW'))
    offH[room] = int(config.get(room, 'offH')) 
    
for room in rooms:
    for anchor in anchors:
        try:
            loc_sensor[anchor]=getLoc(room,anchor,offH[room],offW[room])
        except:
            print("None sensor",room,anchor)
print(loc_sensor)


livingroom
kitchen
bathroom
S: 8171
S: eb38
S: 1fa0
None sensor livingroom 1fa0
S: a304
None sensor livingroom a304
S: c492
None sensor livingroom c492
S: 8171
None sensor kitchen 8171
S: eb38
None sensor kitchen eb38
S: 1fa0
S: a304
S: c492
None sensor kitchen c492
S: 8171
None sensor bathroom 8171
S: eb38
None sensor bathroom eb38
S: 1fa0
None sensor bathroom 1fa0
S: a304
None sensor bathroom a304
S: c492
{'8171': [0, 5], 'eb38': [4, 2], '1fa0': [9, 4], 'a304': [14, 1], 'c492': [10, 8]}


In [ ]:
from datetime import datetime
import os
from tkinter import *
from PIL import ImageTk, Image
import time


d0=40


def getXY(i,j):
    return j*d0+5,i*d0+5

def getFPImg(tt):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0

#def getSceneImg(scene):
#    return np.array(Image.open(data_folder+"/"+room+"/DAY_"+str(day)+"/FP.mean."+str(scene)+".png"))/255.0

def getGTImg(tt,user):
    return np.array(Image.open(data_folder+"/DAY_"+str(day)+"/GT."+user+"."+str(tt)+".png"))/255.0    

def getUserAnchor(user,anchor,tt):
    #print(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
    df_total=read_csv(data_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d", "d2"])
    #print(df_total)
    df_intervalo = df_total[(df_total['time'] >= (tt-1)) & (df_total['time'] <= (tt)) & (df_total['d'] != -1)]
    #print("df_in#tervalo",df_intervalo)
    if(df_intervalo.size>0):
        d=np.mean(df_intervalo['d'])
        #print("d",d)
        #print(d,1-((d - rssi_min) / (rssi_max - rssi_min)))
        #if(d!=-1):
        #    return 1-((d - rssi_min) / (rssi_max - rssi_min))
        if(d<10):
            return (1-d/10.0)*(1-d/10.0)
        else:
            return 0.0
    return -1 
    
    

def getTime():
    return str(int(time.time()))+"\t"+datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def key_file_time():
    return str((int)(time.time()/(60*60*24)))

def key_file_time_now():
    return str((int)(time.time()))

def create_circle(x, y, r, canvasName): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    return canvasName.create_oval(x0, y0, x1, y1, fill="red")

def motion(event):
    x, y = event.x, event.y
    print("Current Position = ", (x, y))
    create_circle(x, y, 25, canvas)

def display_text(text, x, y, tag="myText"):
    """
    Display text on the canvas at the specified (x, y) location.
    If text with the same tag exists, it updates the text instead of creating a new one.
    """
    existing_text = canvas.find_withtag(tag)
    if existing_text:
        canvas.itemconfigure(existing_text, text=text)
        canvas.coords(existing_text, x, y)
    else:
        canvas.create_text(x, y, text=text, fill="black", font=('Helvetica', '12'), tags=(tag,))    
    
def pintar_datos():
    global i0
    
    if(i0>len(ts)):
        return
    
    tt=ts[i0]
    print(tt)

    # Obtener las dimensiones del lienzo
    

    ancho = canvas.winfo_reqwidth()
    alto = canvas.winfo_reqheight()
    



    for user in users:
        for anchor in anchors:
            canvas.delete("point"+user+"x"+anchor)
            
    for i in range(h):  
        for j in range(w): 
            canvas.delete("red"+str(i)+"x"+str(j))
            
    for user in users:
        for i in range(h):  
            for j in range(w): 
                canvas.delete("gt"+user+"x"+str(i)+"x"+str(j))

    fpimg=getFPImg(tt)

    
    #print("fpimg",fpimg)

        

    for user in users:
        gtimg=getGTImg(tt,user)
        for i in range(h):  
            for j in range(w): 
                x,y=getXY(i,j)
                d=d0*gtimg[i][j]
                
                #print("user:",user,i,j,gtimg[i][j],x,y)
                canvas.create_oval(x - d, y - d, x + d, y + d, outline=user_color[user], fill=user_color[user], tags="gt"+user+"x"+str(i)+"x"+str(j))
    
    for i in range(h):  
        for j in range(w): 
            x,y=getXY(i,j)
            #d=d0*(fpimg[i][j]-img00[i][j])
            d=d0*(fpimg[i][j])
            canvas.create_oval(x - d, y - d, x + d, y + d, outline="grey", fill="grey", tags="red"+str(i)+"x"+str(j))

    for user in users:
        for anchor in anchors:
            d=getUserAnchor(user,anchor,tt)*d0
            if(d<0):
                d=2
            x,y=loc_sensor[anchor]
            x,y=getXY(x,y)
            #print(user,anchor,d)
            canvas.create_oval(x - d, y - d, x + d, y + d, outline=user_color[user], tags="point"+user+"x"+anchor)
    display_text("#"+time2str(tt), 50, 5, tag="greeting")
    i0=i0+1
    canvas.after(125, pintar_datos)    

win = Tk()

img = ImageTk.PhotoImage(Image.open("mapa-RADAR-sim.png").resize((w*d0, h*d0)))
canvas = Canvas(win, width=w*d0,height=h*d0)
canvas.create_image(0, 0,anchor=NW, image=img)
canvas.bind('<Button 1>', motion)

canvas.pack()

pintar_datos()


win.mainloop()

315576002
315576003
315576004
315576005
315576006
315576007
315576008
315576009
315576010
315576011
315576012
315576013
315576014
315576015
315576016
315576017
315576018
315576019
315576020
315576021
315576022
315576023
315576024
315576025
315576026
315576027
315576028
315576029
315576030
315576031
315576032
315576033
315576034
315576035
315576036
315576037
315576038
315576039
315576040
315576041
315576042
315576043
315576044
315576045
315576046
315576047
315576048
315576049
315576050
315576051
315576052
315576053
315576054
315576055
315576056
315576057
315576058
315576059
315576060
315576061
315576062
315576063
315576064
315576065
315576066
315576067
315576068
315576069
315576070
315576071
315576072
315576073
315576074
315576075
315576076
315576077
315576078
315576079
315576080
315576081
315576082
315576083
315576084
315576085
315576086
315576087
315576088
315576089
315576090
315576091
315576092
315576093
315576094
315576095
315576096
315576097
315576098
315576099
315576100
315576101


315576823
315576824
315576825
315576826
315576827
315576828
315576829
315576830
315576831
315576832
315576833
315576834
315576835
315576836
315576837
315576838
315576839
315576840
315576841
315576842
315576843
315576844
315576845
315576846
315576847
315576848
315576849
315576850
315576851
315576852
315576853
315576854
315576855
315576856
315576857
315576858
315576859
315576860
315576861
315576862
315576863
315576864
315576865
315576866
315576867
315576868
315576869
315576870
315576871
315576872
315576873
315576874
315576875
315576876
315576877
315576878
315576879
315576880
315576881
315576882
315576883
315576884
315576885
315576886
315576887
315576888
315576889
315576890
315576891
315576892
315576893
315576894
315576895
315576896
315576897
315576898
315576899
315576900
315576901
315576902
315576903
315576904
315576905
315576906
315576907
315576908
315576909
315576910
315576911
315576912
315576913
315576914
315576915
315576916
315576917
315576918
315576919
315576920
315576921
315576922


315577644
315577645
315577646
315577647
315577648
315577649
315577650
315577651
315577652
315577653
315577654
315577655
315577656
315577657
315577658
315577659
315577660
315577661
315577662
315577663
315577664
315577665
315577666
315577667
315577668
315577669
315577670
315577671
315577672
315577673
315577674
315577675
315577676
315577677
315577678
315577679
315577680
315577681
315577682
315577683
315577684
315577685
315577686
315577687
315577688
315577689
315577690
315577691
315577692
315577693
315577694
315577695
315577696
315577697
315577698
315577699
315577700
315577701
315577702
315577703
315577704
315577705
315577706
315577707
315577708
315577709
315577710
315577711
315577712
315577713
315577714
315577715
315577716
315577717
315577718
315577719
315577720
315577721
315577722
315577723
315577724
315577725
315577726
315577727
315577728
315577729
315577730
315577731
315577732
315577733
315577734
315577735
315577736
315577737
315577738
315577739
315577740
315577741
315577742
315577743


315578465
315578466
315578467
315578468
315578469
315578470
315578471
315578472
315578473
315578474
315578475
315578476
315578477
315578478
315578479
315578480
315578481
315578482
315578483
315578484
315578485
315578486
315578487
315578488
315578489
315578490
315578491
315578492
315578493
315578494
315578495
315578496
315578497
315578498
315578499
315578500
315578501
315578502
315578503
315578504
315578505
315578506
315578507
315578508
315578509
315578510
315578511
315578512
315578513
315578514
315578515
315578516
315578517
315578518
315578519
315578520
315578521
315578522
315578523
315578524
315578525
315578526
315578527
315578528
315578529
315578530
315578531
315578532
315578533
315578534
315578535
315578536
315578537
315578538
315578539
315578540
315578541
315578542
315578543
315578544
315578545
315578546
315578547
315578548
315578549
315578550
315578551
315578552
315578553
315578554
315578555
315578556
315578557
315578558
315578559
315578560
315578561
315578562
315578563
315578564


315579285
315579286
315579287
315579288
315579289
315579290
315579291
315579292
315579293
315579294
315579295
315579296
315579297
315579298
315579299
315579300
315579301
315579302
315579303
315579304
315579305
315579306
315579307
315579308
315579309
315579310
315579311
315579312
315579313
315579314
315579315
315579316
315579317
315579318
315579319
315579320
315579321
315579322
315579323
315579324
315579325
315579326
315579327
315579328
315579329
315579330
315579331
315579332
315579333
315579334
315579335
315579336
315579337
315579338
315579339
315579340
315579341
315579342
315579343
315579344
315579345
315579346
315579347
315579348
315579349
315579350
315579351
315579352
315579353
315579354
315579355
315579356
315579357
315579358
315579359
315579360
315579361
315579362
315579363
315579364
315579365
315579366
315579367
315579368
315579369
315579370
315579371
315579372
315579373
315579374
315579375
315579376
315579377
315579378
315579379
315579380
315579381
315579382
315579383
315579384


315580106
315580107
315580108
315580109
315580110
315580111
315580112
315580113
315580114
315580115
315580116
315580117
315580118
315580119
315580120
315580121
315580122
315580123
315580124
315580125
315580126
315580127
315580128
315580129
315580130
315580131
315580132
315580133
315580134
315580135
315580136
315580137
315580138
315580139
315580140
315580141
315580142
315580143
315580144
315580145
315580146
315580147
315580148
315580149
315580150
315580151
315580152
315580153
315580154
315580155
315580156
315580157
315580158
315580159
315580160
315580161
315580162
315580163
315580164
315580165
315580166
315580167
315580168
315580169
315580170
315580171
315580172
315580173
315580174
315580175
315580176
315580177
315580178
315580179
315580180
315580181
315580182
315580183
315580184
315580185
315580186
315580187
315580188
315580189
315580190
315580191
315580192
315580193
315580194
315580195
315580196
315580197
315580198
315580199
315580200
315580201
315580202
315580203
315580204
315580205


315580926
315580927
315580928
315580929
315580930
315580931
315580932
315580933
315580934
315580935
315580936
315580937
315580938
315580939
315580940
315580941
315580942
315580943
315580944
315580945
315580946
315580947
315580948
315580949
315580950
315580951
315580952
315580953
315580954
315580955
315580956
315580957
315580958
315580959
315580960
315580961
315580962
315580963
315580964
315580965
315580966
315580967
315580968
315580969
315580970
315580971
315580972
315580973
315580974
315580975
315580976
315580977
315580978
315580979
315580980
315580981
315580982
315580983
315580984
315580985
315580986
315580987
315580988
315580989
315580990
315580991
315580992
315580993
315580994
315580995
315580996
315580997
315580998
315580999
315581000
315581001
315581002
315581003
315581004
315581005
315581006
315581007
315581008
315581009
315581010
315581011
315581012
315581013
315581014
315581015
315581016
315581017
315581018
315581019
315581020
315581021
315581022
315581023
315581024
315581025


315581746
315581747
315581748
315581749
315581750
315581751
315581752
315581753
315581754
315581755
315581756
315581757
315581758
315581759
315581760
315581761
315581762
315581763
315581764
315581765
315581766
315581767
315581768
315581769
315581770
315581771
315581772
315581773
315581774
315581775
315581776
315581777
315581778
315581779
315581780
315581781
315581782
315581783
315581784
315581785
315581786
315581787
315581788
315581789
315581790
315581791
315581792
315581793
315581794
315581795
315581796
315581797
315581798
315581799
315581800
315581801
315581802
315581803
315581804
315581805
315581806
315581807
315581808
315581809
315581810
315581811
315581812
315581813
315581814
315581815
315581816
315581817
315581818
315581819
315581820
315581821
315581822
315581823
315581824
315581825
315581826
315581827
315581828
315581829
315581830
315581831
315581832
315581833
315581834
315581835
315581836
315581837
315581838
315581839
315581840
315581841
315581842
315581843
315581844
315581845


315582566
315582567
315582568
315582569
315582570
315582571
315582572
315582573
315582574
315582575
315582576
315582577
315582578
315582579
315582580
315582581
315582582
315582583
315582584
315582585
315582586
315582587
315582588
315582589
315582590
315582591
315582592
315582593
315582594
315582595
315582596
315582597
315582598
315582599
315582600
315582601
315582602
315582603
315582604
315582605
315582606
315582607
315582608
315582609
315582610
315582611
315582612
315582613
315582614
315582615
315582616
315582617
315582618
315582619
315582620
315582621
315582622
315582623
315582624
315582625
315582626
315582627
315582628
315582629
315582630
315582631
315582632
315582633
315582634
315582635
315582636
315582637
315582638
315582639
315582640
315582641
315582642
315582643
315582644
315582645
315582646
315582647
315582648
315582649
315582650
315582651
315582652
315582653
315582654
315582655
315582656
315582657
315582658
315582659
315582660
315582661
315582662
315582663
315582664
315582665


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\javie\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\javie\anaconda3\lib\tkinter\__init__.py", line 814, in callit
    func(*args)
  File "C:\Users\javie\AppData\Local\Temp\ipykernel_37612\3051407136.py", line 82, in pintar_datos
    tt=ts[i0]
IndexError: list index out of range
